Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Path definition.

In [2]:
dataset = 'CIFAR-100'
current_directory = os.getcwd()
csv_path = os.path.join(current_directory, 'result', dataset, 'total_average_scores.csv')


save_path = os.path.join(current_directory, 'result_processed', dataset)
os.makedirs(save_path, exist_ok=True)
result_path = os.path.join(save_path, 'scores_compared.csv')
plot_path = os.path.join(save_path, 'groupped_figure.png')

Load data as dataframe

In [3]:
scores_df = pd.read_csv(csv_path)
scores_df.head()

,gof_scores,Bottleneck2-1,Bottleneck2-2,Bottleneck2-3,Bottleneck3-1,Bottleneck3-2,Bottleneck3-3,Bottleneck3-4,Bottleneck4-1,Bottleneck4-2,Bottleneck4-3,Bottleneck4-4,Bottleneck4-5,Bottleneck4-6,Bottleneck5-1,Bottleneck5-2,Bottleneck5-3,overall
0,gaussian_DG,-0.975087,-0.983500,-0.973953,-0.975032,-0.973483,-0.972873,-0.973961,-0.877193,-0.878504,-0.874792,-0.880247,-0.878147,-0.894459,-0.804528,-0.813956,-0.858906,-0.911789
1,laplace_DG,-1.013139,-1.018511,-1.014007,-0.940198,-0.944358,-0.952415,-0.951426,-0.874719,-0.885953,-0.888268,-0.895177,-0.887369,-0.891455,-0.850814,-0.842497,-0.849738,-0.918753
2,gaussian_KGE,-0.001423,-0.008649,-0.000941,-0.001016,-0.000707,-0.000463,-0.000483,-0.006972,-0.011440,-0.006746,-0.009843,-0.009952,-0.020125,-0.062132,-0.120865,-0.147339,-0.025568
3,laplace_KGE,-0.000434,-0.003343,-0.000867,0.005416,0.002933,-0.001243,-0.000343,0.000839,-0.021689,-0.011149,-0.015595,-0.017006,-0.025708,-0.049934,-0.069617,-0.075404,-0.017697
4,gaussian_NSE,-0.975087,-0.983500,-0.973953,-0.975032,-0.973483,-0.972873,-0.973961,-0.877193,-0.878504,-0.874792,-0.880247,-0.878147,-0.894459,-0.804528,-0.813956,-0.858906,-0.911789


Discard no need columns

In [4]:
overall_df = scores_df.iloc[:, [0, -1]]
overall_df.head()

,gof_scores,overall
0,gaussian_DG,-0.911789
1,laplace_DG,-0.918753
2,gaussian_KGE,-0.025568
3,laplace_KGE,-0.017697
4,gaussian_NSE,-0.911789


In [ ]:
# Step 1: Split 'gof_scores' and expand into two new columns
overall_df[['distribution', 'metric']] = overall_df['gof_scores'].str.split('_', n=1, expand=True)

# Step 2: Pivot the DataFrame to get 'distribution' as rows and 'metric' as columns
pivot_df = overall_df.pivot(index='distribution', columns='metric', values='overall')

# Resetting the index to bring 'distribution' back as a column
pivot_df.reset_index(inplace=True)
pivot_df.head()

/tmp/ipykernel_20760/2348768855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overall_df[['distribution', 'metric']] = overall_df['gof_scores'].str.split('_', n=1, expand=True)
/tmp/ipykernel_20760/2348768855.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overall_df[['distribution', 'metric']] = overall_df['gof_scores'].str.split('_', n=1, expand=True)


metric,distribution,DG,KGE,NSE,d,mNSE,mae,md,me,nrmse,r2,rNSE,rSD,r_pearson,rd,rmse,sdr
0,gaussian,-0.911789,-0.025568,-0.911789,0.354708,-0.591365,0.079216,0.281812,-0.003079,1.381428,0.006428,-2.495569e+13,0.951604,0.001415,-7.951932e+12,0.103073,0.102885
1,laplace,-0.918753,-0.017697,-0.918753,0.336236,-0.550880,0.077778,0.269623,-0.003731,1.384173,0.005108,-2.497963e+13,0.955435,-0.000028,-8.196823e+12,0.103174,0.103052


In [6]:
pivot_df.shape

(2, 17)

**Reference**: [ChatGPT Answer](https://chat.openai.com/share/dd7dddc6-533e-4039-8b30-9ca6b5614029)

How to evaluate each metric:


- **Point-to-point**: 

    a larger value is better.
- **Two-Sample Anderson-Darling**:  

    a smaller value is better.
- **Two-Sample Kolmogorov-Smirnov**: 

    a smaller value is better.
- **Deviation of gain**: 

    a smaller value is better.
- **Mean Error**: 
    
    A smaller absolute value is better.
- **Mean Absolute Error**: 

    a smaller value is better.
- **Root Mean Square Error**: 
    
    a smaller value is better.
- **Normalized Root Mean Square Error**: 

    a smaller value is better.
- **Standard deviation of residual**: 

    a smaller value is better.
- **Kling Gupta Efficiency**: 

    A larger value is better. 1 indicates a perfect match.A smaller value (especially negative values) indicates a poor fit. 
- **Nash-sutcliffe Efficiency**:

    A larger value is better. 1 indicates a perfect match.A smaller value (especially negative values) indicates a poor fit.
- **Modified Nash-sutcliffe Efficiency**:

    A larger value is better. 1 signifies perfect agreement.
- **Relative Nash-sutcliffe Efficiency**:

    A larger value is better. 1 signifies perfect agreement. -inf indicates an extreme case where the model's predictions are significantly worse than the mean of the observed data. 
- **Coefficient of Determination**: 

    A larger value is better. 1 signifies perfect agreement.
- **Ratio of Standard Deviations**:

    A value close to 1 is ideal.
- **Pearson product-moment correlation coefficient**:

    A higher value closer to 1 is preferable as it indicates a stronger positive linear relationship between the observed data and the predictions from the fitted PDF, suggesting a good fit.
- **Index of Agreement**:

    A larger value is better. 1 signifies perfect agreement.
- **Modified Index of Agreement**:

    A larger value is better. 1 signifies perfect agreement.
- **Relative Index of Agreement**:

    A larger value is better. 1 signifies perfect agreement. A result of -inf in the context of the Relative Index of Agreement is highly unusual and typically indicates a computational or interpretational anomaly. 



In [7]:
name_dict = {
    'ADTestTwoSampleGOF': 'Two-Sample Anderson-Darling',
    'KSTestTwoSampleGOF': 'Two-Sample Kolmogorov-Smirnov',
    'PointToPointGOF': 'Point-to-point',
    'DG': 'Deviation of gain',
    'KGE': 'Kling Gupta Efficiency',
    'NSE': 'Nash-sutcliffe Efficiency',
    'd': 'Index of Agreement',
    'mNSE': 'Modified Nash-sutcliffe Efficiency',
    'mae': 'Mean Absolute Error',
    'md': 'Modified Index of Agreement',
    'me': 'Mean Error',
    'nrmse': 'Normalized Root Mean Square Error',
    'r2': 'Coefficient of Determination',
    'rNSE': 'Relative Nash-sutcliffe Efficiency',
    'rSD': 'Ratio of Standard Deviations',
    'r_pearson': 'Pearson product-moment correlation coefficient',
    'rd': 'Relative Index of Agreement',
    'rmse': 'Root Mean Square Error',
    'sdr': 'Standard deviation of residual'
}

sort_name = [
    'Two-Sample Anderson-Darling', 'Two-Sample Kolmogorov-Smirnov', 'Point-to-point',
    'Deviation of gain', 'Standard deviation of residual',
    'Mean Error', 'Mean Absolute Error', 'Root Mean Square Error', 'Normalized Root Mean Square Error',
    'Index of Agreement', 'Modified Index of Agreement', 'Relative Index of Agreement',
    'Nash-sutcliffe Efficiency', 'Modified Nash-sutcliffe Efficiency', 'Relative Nash-sutcliffe Efficiency',
    'Kling Gupta Efficiency', 'Coefficient of Determination', 'Pearson product-moment correlation coefficient', 'Ratio of Standard Deviations',
]

small_better = [
    'Two-Sample Anderson-Darling', 'Two-Sample Kolmogorov-Smirnov',
    'Deviation of gain', 'Standard deviation of residual',
    'Mean Error', 'Mean Absolute Error', 'Root Mean Square Error', 'Normalized Root Mean Square Error',
]

large_better = [
    'Point-to-point',
]

abs1_better = [
    'Index of Agreement', 'Modified Index of Agreement', 'Relative Index of Agreement',
    'Nash-sutcliffe Efficiency', 'Modified Nash-sutcliffe Efficiency', 'Relative Nash-sutcliffe Efficiency',
    'Kling Gupta Efficiency', 'Coefficient of Determination', 'Pearson product-moment correlation coefficient', 'Ratio of Standard Deviations',
]

In [8]:
# Step 1: Rename columns in pivot_df according to name_dict
pivot_df.rename(columns=name_dict, inplace=True)

# Step 2: Reorder columns according to sort_name, keeping 'distribution' as the first column
# Note: Ensure that 'distribution' is in the first column and all other metric names are in sort_name
ordered_columns = ['distribution'] + [col for col in sort_name if col in pivot_df.columns]

# Reassign the columns of pivot_df based on the new order
pivot_df = pivot_df[ordered_columns]

In [9]:
pivot_df.head()

metric,distribution,Deviation of gain,Standard deviation of residual,Mean Error,Mean Absolute Error,Root Mean Square Error,Normalized Root Mean Square Error,Index of Agreement,Modified Index of Agreement,Relative Index of Agreement,Nash-sutcliffe Efficiency,Modified Nash-sutcliffe Efficiency,Relative Nash-sutcliffe Efficiency,Kling Gupta Efficiency,Coefficient of Determination,Pearson product-moment correlation coefficient,Ratio of Standard Deviations
0,gaussian,-0.911789,0.102885,-0.003079,0.079216,0.103073,1.381428,0.354708,0.281812,-7.951932e+12,-0.911789,-0.591365,-2.495569e+13,-0.025568,0.006428,0.001415,0.951604
1,laplace,-0.918753,0.103052,-0.003731,0.077778,0.103174,1.384173,0.336236,0.269623,-8.196823e+12,-0.918753,-0.550880,-2.497963e+13,-0.017697,0.005108,-0.000028,0.955435


In [10]:
print(pivot_df.index.duplicated())

[False False]


In [11]:
# Find the index for 'gaussian' and 'laplace' rows
gaussian_row = pivot_df[pivot_df['distribution'] == 'gaussian'].iloc[0]
laplace_row = pivot_df[pivot_df['distribution'] == 'laplace'].iloc[0]

# Initialize a new row with 'laplace_better' in the 'distribution' column
new_row = {'distribution': 'laplace_better'}
# Loop through each column from the second to the last
for col in pivot_df.columns[1:]:
    gaussian_value = gaussian_row[col]
    laplace_value = laplace_row[col]

    # Initialize the new row value as empty
    new_row[col] = np.nan  # Use np.nan for "empty"

    # Apply the 'small_better' rule
    if col in small_better and laplace_value < gaussian_value:
        new_row[col] = True

    elif col in large_better and laplace_value > gaussian_value:
        new_row[col] = True

    # Apply the 'abs1_better' rule
    elif col in abs1_better and abs(laplace_value - 1) < abs(gaussian_value - 1):
        new_row[col] = True

# Convert new_row dictionary to DataFrame
new_row_df = pd.DataFrame([new_row])

# Use pd.concat to append new_row_df to pivot_df
pivot_df = pd.concat([pivot_df, new_row_df], ignore_index=True)

In [12]:
# Initialize the new row with 'distribution' column set to 'standard'
new_row = {'distribution': 'standard'}

# Iterate over the columns in pivot_df except for 'distribution'
for col in [c for c in pivot_df.columns if c != 'distribution']:
    if col in small_better:
        new_row[col] = 'small better'
    elif col in large_better:
        new_row[col] = 'large better'
    elif col in abs1_better:
        new_row[col] = 'close to 1 better'
    else:
        new_row[col] = ''

# Convert the new_row dictionary to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Append new_row_df to pivot_df
pivot_df = pd.concat([pivot_df, new_row_df], ignore_index=True)

# Ensure that 'distribution' column remains a column and not an index after appending
if 'distribution' not in pivot_df.columns:
    pivot_df.reset_index(inplace=True)
    pivot_df.rename(columns={'index': 'distribution'}, inplace=True)

In [13]:
pivot_df.head()

,distribution,Deviation of gain,Standard deviation of residual,Mean Error,Mean Absolute Error,Root Mean Square Error,Normalized Root Mean Square Error,Index of Agreement,Modified Index of Agreement,Relative Index of Agreement,Nash-sutcliffe Efficiency,Modified Nash-sutcliffe Efficiency,Relative Nash-sutcliffe Efficiency,Kling Gupta Efficiency,Coefficient of Determination,Pearson product-moment correlation coefficient,Ratio of Standard Deviations
0,gaussian,-0.911789,0.102885,-0.003079,0.079216,0.103073,1.381428,0.354708,0.281812,-7951932180685.730469,-0.911789,-0.591365,-24955687904679.507812,-0.025568,0.006428,0.001415,0.951604
1,laplace,-0.918753,0.103052,-0.003731,0.077778,0.103174,1.384173,0.336236,0.269623,-8196822904283.748047,-0.918753,-0.55088,-24979631614703.601562,-0.017697,0.005108,-0.000028,0.955435
2,laplace_better,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
3,standard,small better,small better,small better,small better,small better,small better,close to 1 better,close to 1 better,close to 1 better,close to 1 better,close to 1 better,close to 1 better,close to 1 better,close to 1 better,close to 1 better,close to 1 better


In [14]:
pivot_df.to_csv(result_path, index=False)